In [11]:
# imports
import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np
import math
import json
import matplotlib.pyplot as plt
from preprocessing_tools import check_for_nan_columns, drop_nan_columns
from functools import partial
from scipy import stats
from scipy.stats import randint

from sklearn.model_selection import (train_test_split, cross_val_score, 
                                     GridSearchCV, RandomizedSearchCV)
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import mean_squared_error, make_scorer

In [34]:
df = pd.read_parquet('NYSE_dataset_v2_linregress.parquet')
df = df.drop(['priceRatioRelativeToS&P_1Q', 'priceRatioRelativeToS&P_3Q', 
              'priceRatioRelativeToS&P_4Q'], axis=1)

# making the output binary
df['priceRatioRelativeToS&P_2Q'] = \
    df['priceRatioRelativeToS&P_2Q'].apply(lambda x: '1' if x>=1 else '0')

input_df = df.drop('priceRatioRelativeToS&P_2Q', axis=1).copy()
output_df = df['priceRatioRelativeToS&P_2Q'].copy().to_frame()

X_train, X_test, y_train, y_test = train_test_split(input_df, 
                                                    output_df,
                                                    random_state=42,
                                                    test_size=0.2
                                                   )

# scaling data
input_scaler = StandardScaler()
X_train_scaled = input_scaler.fit_transform(X_train)
X_test_scaled = input_scaler.transform(X_test)
# categorical outputs dont need to be scaled
np.array(y_train).ravel().shape

(25744,)

## Stochastic Gradient Descent

In [43]:
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train_scaled, np.array(y_train).ravel())

SGDClassifier(random_state=42)